# Imports

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
from google.colab import files
from tqdm.notebook import tqdm

#Pipeline de procesado

## Subir el csv

In [39]:
uploaded = files.upload()
for filename in uploaded.keys():
  dataframe = pd.read_csv(filename)

Saving russian_df_part_5.csv to russian_df_part_5.csv


In [40]:
  dataframe.head()

,id,review,type_review,date,language,views,votes,score
0,Portos,"Кошмар , открытая пропаганда лгбт и феминизма...",expanded,2020-06-21,Russian,5,4,0
1,MadWulff,"Создатели, вы в своем вообще уме??? заставить ...",normal,2020-06-21,Russian,5,4,0
2,googlogmob,Игра потеряла героев с первой части. Во второй...,normal,2020-06-21,Russian,4,3,0
3,broukers,"Разработчики перешли черту, это просто неподоб...",normal,2020-06-21,Russian,11,8,0
4,AweZ,"Игра очень разочеровала, начала и конец игры э...",normal,2020-06-21,Russian,16,11,0


## Calcular sentiment análisis

In [41]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from scipy.special import softmax

MODEL_NAME = "seara/rubert-tiny2-russian-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = softmax(outputs.logits[0].numpy())

    labels = ["neutral", "positive", "negative"]

    results = {label: probability for label, probability in zip(labels, probabilities)}

    return results



reviews = dataframe['review']

for index, text in tqdm(enumerate(reviews), total=len(reviews)):
  results = analyze_sentiment(text)
  for label, score in results.items():
    dataframe.at[index, label] = score


  0%|          | 0/247 [00:00<?, ?it/s]

In [42]:
dataframe.head()

,id,review,type_review,date,language,views,votes,score,neutral,positive,negative
0,Portos,"Кошмар , открытая пропаганда лгбт и феминизма...",expanded,2020-06-21,Russian,5,4,0,0.436036,0.014254,0.549710
1,MadWulff,"Создатели, вы в своем вообще уме??? заставить ...",normal,2020-06-21,Russian,5,4,0,0.050158,0.004258,0.945584
2,googlogmob,Игра потеряла героев с первой части. Во второй...,normal,2020-06-21,Russian,4,3,0,0.831897,0.015177,0.152926
3,broukers,"Разработчики перешли черту, это просто неподоб...",normal,2020-06-21,Russian,11,8,0,0.338339,0.020953,0.640708
4,AweZ,"Игра очень разочеровала, начала и конец игры э...",normal,2020-06-21,Russian,16,11,0,0.114788,0.009917,0.875294


## Descargar los resultados de salida

In [43]:
filename = f'sent_russian_part_4.csv'
dataframe.to_csv(filename, index=False)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>